# SBERT

Implementation of the following papers:

BERT: https://arxiv.org/pdf/1810.04805.pdf
SBERT: https://arxiv.org/pdf/1908.10084v1.pdf

In [1]:
import os
import numpy as np
import pandas as pd
import logging
from preprocess import Preprocess
import torch
import numpy as np
import sts_data
from sts_data import STSData
from importlib import reload
from transformers import BertModel
from train import train_model
from evaluation import evaluate_test_set

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
stopwords_path="stopwords-en.txt"
dataset_name = "sick"
batch_size = 64

reload(sts_data)

dataset_name = "sick"

sbert_data = STSData(
    dataset_name=dataset_name,
)

batch_size = 64

## fetching dataloaders
sbert_dataloaders = sbert_data.get_data_loader(batch_size=batch_size)

INFO:root:loading and preprocessing data...
INFO:root:reading and preprocessing data completed...


INFO:root:creating dataloaders completed...


In [4]:
# importing pre-trained BERT base model
model = BertModel.from_pretrained('bert-base-uncased')

# setting Adam optimizer with learning rate as specified by SBERT paper
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

max_epochs = 1

train_acc, train_loss = train_model(model, optimizer, sbert_dataloaders['train'], max_epochs)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [19:06<00:00, 16.38s/it]
INFO:root:Accuracy: (0.734628916427

In [5]:
print(train_loss/len(sbert_dataloaders['train']))

0.019411560547138964


In [6]:
model_path = './sbert_fine_tuned'

if not os.path.exists(model_path):
    os.mkdir(model_path)

model.save_pretrained(model_path)

In [7]:
test_model = BertModel.from_pretrained(model_path)

evaluate_test_set(test_model,sbert_dataloaders['test'])

100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [14:10<00:00, 11.04s/it]


Accuracy: 0.7770470357342115 


# References:

1. "Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks", Nils Reimers and Iryna Gurevych
2. "Training Sentence Transformers the OG Way (with Softmax Loss)", Pinecone
    https://www.pinecone.io/learn/train-sentence-transformers-softmax/
3. https://medium.com/swlh/transformer-based-sentence-embeddings-cd0935b3b1e0


Library used for pre-trained model:
https://www.sbert.net